## **1) Importing Python Packages for GAN**


In [1]:
from keras.datasets import mnist

from keras.models import Sequential
from keras.layers import BatchNormalization
from keras.layers import Dense, Reshape, Flatten
from keras.layers.advanced_activations import LeakyReLU
from tensorflow.keras.optimizers import Adam

import numpy as np
!mkdir generated_images

## **2) Variables for Neural Networks & Data**

In [2]:
img_width = 28
img_height = 28
channels = 1
img_shape = (img_width, img_height, channels)
latent_dim = 100
adam = Adam(lr=0.0001)

/opt/homebrew/Caskroom/miniforge/base/envs/lhl_env38/lib/python3.8/site-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


## **3) Building Generator**





In [3]:
def build_generator():
  model = Sequential()

  model.add(Dense(256, input_dim=latent_dim))
  model.add(LeakyReLU(alpha=0.2))
  model.add(BatchNormalization(momentum=0.8))

  model.add(Dense(256))
  model.add(LeakyReLU(alpha=0.2))
  model.add(BatchNormalization(momentum=0.8))

  model.add(Dense(256))
  model.add(LeakyReLU(alpha=0.2))
  model.add(BatchNormalization(momentum=0.8))

  model.add(Dense(np.prod(img_shape), activation='tanh'))
  model.add(Reshape(img_shape))
  return model

generator = build_generator()

## **4) Building Discriminator**

In [4]:
def build_discriminator():
  model = Sequential()

  model.add(Flatten(input_shape=img_shape))
  model.add(Dense(512))
  model.add(LeakyReLU(alpha=0.2))
  model.add(Dense(256))
  model.add(Dense(1, activation='sigmoid'))

  model.summary()
  return model

discriminator = build_discriminator()
discriminator.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense_4 (Dense)             (None, 512)               401920    
                                                                 
 leaky_re_lu_3 (LeakyReLU)   (None, 512)               0         
                                                                 
 dense_5 (Dense)             (None, 256)               131328    
                                                                 
 dense_6 (Dense)             (None, 1)                 257       
                                                                 
Total params: 533,505
Trainable params: 533,505
Non-trainable params: 0
_________________________________________________________________


## **5) Connecting Neural Networks to build GAN**

In [5]:
GAN = Sequential()
discriminator.trainable = False
GAN.add(generator)
GAN.add(discriminator)

GAN.compile(loss='binary_crossentropy', optimizer=adam)
GAN.summary()


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential (Sequential)     (None, 28, 28, 1)         362000    
                                                                 
 sequential_1 (Sequential)   (None, 1)                 533505    
                                                                 
Total params: 895,505
Trainable params: 360,464
Non-trainable params: 535,041
_________________________________________________________________


## **6) Outputting Images**


In [6]:
#@title
## **7) Outputting Images**
import matplotlib.pyplot as plt
import glob
import imageio
import PIL

save_name = 0.00000000

def save_imgs(epoch):
    r, c = 5, 5
    noise = np.random.normal(0, 1, (r * c, latent_dim))
    gen_imgs = generator.predict(noise)
    global save_name
    save_name += 0.00000001
    print("%.8f" % save_name)

    # Rescale images 0 - 1
    gen_imgs = 0.5 * gen_imgs + 0.5

    fig, axs = plt.subplots(r, c)
    cnt = 0
    for i in range(r):
        for j in range(c):
            axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
            # axs[i,j].imshow(gen_imgs[cnt])
            axs[i,j].axis('off')
            cnt += 1
    fig.savefig("generated_images/%.8f.png" % save_name)
    print('saved')
    plt.close()

## **7) Training GAN**

In [8]:
def train(epochs, batch_size=64, save_interval=200):
  (X_train, _), (_, _) = mnist.load_data()

  # print(X_train.shape)
  #Rescale data between -1 and 1, currently and 3 dimensions (w,l,color)
  X_train = X_train / 127.5 -1.
  # X_train = np.expand_dims(X_train, axis=3)
  # print(X_train.shape)

  #Create our Y for our Neural Networks - # creating a loss
  valid = np.ones((batch_size, 1))
  fakes = np.zeros((batch_size, 1))

  for epoch in range(epochs):
    #Get Random Batch - initializing a seed
    idx = np.random.randint(0, X_train.shape[0], batch_size)
    imgs = X_train[idx]

    #Generate Fake Images
    noise = np.random.normal(0, 1, (batch_size, latent_dim))
    gen_imgs = generator.predict(noise)

    #Train discriminator
    d_loss_real = discriminator.train_on_batch(imgs, valid)
    d_loss_fake = discriminator.train_on_batch(gen_imgs, fakes)
    d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

    noise = np.random.normal(0, 1, (batch_size, latent_dim)) # reset noise
    
    #inverse y label
    g_loss = GAN.train_on_batch(noise, valid)

    print("******* %d [D loss: %f, acc: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100* d_loss[1], g_loss))

    if(epoch % save_interval) == 0:
      save_imgs(epoch)

  # print(valid)


train(1000, batch_size=64, save_interval=100)

******* 0 [D loss: 0.011841, acc: 100.00%] [G loss: 4.001544]
0.00000011
saved
******* 1 [D loss: 0.012337, acc: 100.00%] [G loss: 4.036134]
******* 2 [D loss: 0.011586, acc: 100.00%] [G loss: 3.979584]
******* 3 [D loss: 0.011705, acc: 100.00%] [G loss: 4.024421]
******* 4 [D loss: 0.013163, acc: 100.00%] [G loss: 3.961791]
******* 5 [D loss: 0.010247, acc: 100.00%] [G loss: 4.039857]
******* 6 [D loss: 0.010104, acc: 100.00%] [G loss: 4.014960]
******* 7 [D loss: 0.011210, acc: 100.00%] [G loss: 4.048666]
******* 8 [D loss: 0.009536, acc: 100.00%] [G loss: 4.018991]
******* 9 [D loss: 0.012399, acc: 100.00%] [G loss: 4.098874]
******* 10 [D loss: 0.009998, acc: 100.00%] [G loss: 4.004060]
******* 11 [D loss: 0.011881, acc: 100.00%] [G loss: 4.107146]
******* 12 [D loss: 0.012763, acc: 100.00%] [G loss: 4.159957]
******* 13 [D loss: 0.009360, acc: 100.00%] [G loss: 4.123012]
******* 14 [D loss: 0.010094, acc: 100.00%] [G loss: 4.078710]
******* 15 [D loss: 0.011355, acc: 100.00%] [G l

### **8) Making GIF**

In [9]:
# Display a single image using the epoch number
# def display_image(epoch_no):
#   return PIL.Image.open('generated_images/%.8f.png'.format(epoch_no))

anim_file = 'dcgan.gif'

with imageio.get_writer(anim_file, mode='I') as writer:
  filenames = glob.glob('generated_images/*.png')
  filenames = sorted(filenames)
  for filename in filenames:
    image = imageio.imread(filename)
    writer.append_data(image)
  image = imageio.imread(filename)
  writer.append_data(image)

/var/folders/fm/m7sbfrtn37dcz2lvxhvj6q0m0000gn/T/ipykernel_10926/1295497862.py:11: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning dissapear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imageio.imread(filename)
/var/folders/fm/m7sbfrtn37dcz2lvxhvj6q0m0000gn/T/ipykernel_10926/1295497862.py:13: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning dissapear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imageio.imread(filename)
